In [1]:
import numpy as np
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="4"
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [3]:
import PIL
from PIL import Image
import tensorflow as tf
import random
import re
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.python.framework.ops import enable_eager_execution
#disable_eager_execution()
enable_eager_execution()

In [4]:
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN

In [5]:
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow_datasets as tfds

In [6]:
from __future__ import print_function
import argparse
from tensorflow.keras.layers import Input
import scipy.misc

from configs import bcolors
from utils import *
import tensorflow_model_optimization as tfmot

In [7]:
from keras_vggface.vggface import VGGFace
from keras.engine import  Model
from keras.layers import Input
from keras.preprocessing import image
from keras.layers import Flatten, Dense, Input
from keras_vggface import utils

In [8]:
import tarfile

## Load Model

In [9]:
vgg_model = VGGFace(model='resnet50', input_shape=(224, 224, 3))
vgg_model.summary()

Model: "vggface_resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
___________________________________________________________________________________

In [10]:
q_model = tfmot.quantization.keras.quantize_model(vgg_model)

In [11]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = np.asarray(image)
	return face_array
 
# load the photo and extract the face
pixels = extract_face('rdj.jpg')

pixels = pixels.astype('float32')
samples = np.expand_dims(pixels, axis=0)
# prepare the face for the model, e.g. center pixels
samples = utils.preprocess_input(samples, version=2)
# create a vggface model
model = VGGFace(model='resnet50')
# perform prediction
yhat = model.predict(samples)
# convert prediction into names
results = utils.decode_predictions(yhat)
# display most likely results
for result in results[0]:
	print('%s: %.3f%%' % (result[0], result[1]*100))

b' Robert_Downey_Jr.': 99.770%
b' Emre_Altu\xc4\x9f': 0.052%
b' Patrick_Timsit': 0.013%
b' Stefano_Accorsi': 0.006%
b' Vittorio_Grigolo': 0.005%


In [12]:
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = np.asarray(image)
	return face_array
 
# load the photo and extract the face
pixels = extract_face('rdj.jpg')

pixels = pixels.astype('float32')
samples = np.expand_dims(pixels, axis=0)
# prepare the face for the model, e.g. center pixels
samples = utils.preprocess_input(samples, version=2)
# perform prediction
yhat = q_model.predict(samples)
# convert prediction into names
results = utils.decode_predictions(yhat)
# display most likely results
for result in results[0]:
	print('%s: %.3f%%' % (result[0], result[1]*100))

b' Thierry_Omeyer': 0.060%
b' Gerald_Henderson': 0.060%
b' Fr\xc3\xa9d\xc3\xa9ric_Michalak': 0.060%
b' Fr\xc3\xa9d\xc3\xa9rick_Bousquet': 0.060%
b' Tom_Hopper': 0.060%


In [100]:
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    pixels = filename
    pixels = pixels.astype(np.uint8)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array

In [11]:
def preprocess_image(features):
    """Preprocesses the given image.
       will convert the images from RGB to BGR, then will zero-center each color channel with respect to the ImageNet dataset, without scaling.
       mean = [103.939, 116.779, 123.68]
       std = None
  """
    image = features["image"]
    image = tf.image.resize(image,[224,224])
    image = tf.keras.applications.resnet.preprocess_input(image)
    
    features["image"] = image
    
    
    return features["image"], features["label"]

In [12]:
data_ds = tfds.load('vgg_face2', split='test[40%:]',data_dir='/local/rcs/asa97/vggface2/VGG-Face2/data/')

In [16]:
train_ds = data_ds.map(preprocess_image).batch(20)

In [17]:
q_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=2e-5),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [18]:
q_model.fit(train_ds,epochs=2)

Epoch 1/2
1926/6776 [=======>......................] - ETA: 50:11 - loss: nan - accuracy: 0.0025

KeyboardInterrupt: 

In [ ]:
q_model.save("./q_model_40_vggface.h5")